# Variaciones Information Value

En esta notebook vemos las distintas variantes de Information Value y cuánto laburo extra demandarían

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("../data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("../output/provinces_words.csv", index_col=0)

cant_palabras = [c for c in df.columns if re.match(r'.*ocurrencias$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*usuarios$', c)]

print("columnas de palabras => {}\n".format(cant_palabras))
print("columnas de usuarios => {}".format(cant_personas))

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)
df["cant_palabra"] = df[cant_palabras].sum(axis=1)
df["cant_usuarios"] = df[cant_personas].sum(axis=1)

columnas de palabras => ['buenosaires_ocurrencias', 'catamarca_ocurrencias', 'chaco_ocurrencias', 'chubut_ocurrencias', 'cordoba_ocurrencias', 'corrientes_ocurrencias', 'entrerios_ocurrencias', 'formosa_ocurrencias', 'jujuy_ocurrencias', 'lapampa_ocurrencias', 'larioja_ocurrencias', 'mendoza_ocurrencias', 'misiones_ocurrencias', 'neuquen_ocurrencias', 'rionegro_ocurrencias', 'salta_ocurrencias', 'sanjuan_ocurrencias', 'sanluis_ocurrencias', 'santacruz_ocurrencias', 'santafe_ocurrencias', 'santiago_ocurrencias', 'tierradelfuego_ocurrencias', 'tucuman_ocurrencias']

columnas de usuarios => ['buenosaires_usuarios', 'catamarca_usuarios', 'chaco_usuarios', 'chubut_usuarios', 'cordoba_usuarios', 'corrientes_usuarios', 'entrerios_usuarios', 'formosa_usuarios', 'jujuy_usuarios', 'lapampa_usuarios', 'larioja_usuarios', 'mendoza_usuarios', 'misiones_usuarios', 'neuquen_usuarios', 'rionegro_usuarios', 'salta_usuarios', 'sanjuan_usuarios', 'sanluis_usuarios', 'santacruz_usuarios', 'santafe_usuario

In [4]:
df = df[(df.cant_palabra > 80) & (df.cant_usuarios >= 5) ]

Calculemos el valor de la información de cada palabra.

In [5]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log(23) - df["entropy_palabras"]
df["delta_personas"] = np.log(23) - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]
df["ival_norm"] = (1+df["ival_palabras"]) * (1+df["ival_personas"])

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [6]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [7]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 161
Total analizadas actualmente = 744
Faltan etiquetar 256 palabras
Porcentaje candidatas = 0.2163978494623656
########################################
Métrica: Personas
Palabras candidatas = 190
Total analizadas actualmente = 709
Faltan etiquetar 291 palabras
Porcentaje candidatas = 0.2679830747531735
########################################
Métrica: PalPer
Palabras candidatas = 181
Total analizadas actualmente = 755
Faltan etiquetar 245 palabras
Porcentaje candidatas = 0.23973509933774834
Faltan etiquetar (sin repeticiones) = 473
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1161
Total analizadas actualmente = 2985
Faltan etiquetar 2015 palabras
Porcentaje candidatas = 0.38894472361809046
########################################
Métrica: PalPer
Palabras candidatas = 1050
Total analizadas actualmente = 2883
Faltan etiquetar 

## Sin usar el log(23)

In [8]:
from contrastes.information_value import shuffled_entropy
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 162
Total analizadas actualmente = 743
Faltan etiquetar 257 palabras
Porcentaje candidatas = 0.2180349932705249
########################################
Métrica: Personas
Palabras candidatas = 213
Total analizadas actualmente = 732
Faltan etiquetar 268 palabras
Porcentaje candidatas = 0.29098360655737704
########################################
Métrica: PalPer
Palabras candidatas = 200
Total analizadas actualmente = 786
Faltan etiquetar 214 palabras
Porcentaje candidatas = 0.2544529262086514
Faltan etiquetar (sin repeticiones) = 461
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1283
Total analizadas actualmente = 3132
Faltan etiquetar 1868 palabras
Porcentaje candidatas = 0.40964240102171134
########################################
Métrica: PalPer
Palabras candidatas = 1169
Total analizadas actualmente = 3081
Faltan etiquetar 

## Usando cálculo con normalización "vieja"


In [10]:
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)


In [11]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [12]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 247
Total analizadas actualmente = 806
Faltan etiquetar 194 palabras
Porcentaje candidatas = 0.3064516129032258
########################################
Métrica: Personas
Palabras candidatas = 260
Total analizadas actualmente = 794
Faltan etiquetar 206 palabras
Porcentaje candidatas = 0.327455919395466
########################################
Métrica: PalPer
Palabras candidatas = 305
Total analizadas actualmente = 936
Faltan etiquetar 64 palabras
Porcentaje candidatas = 0.32585470085470086
Faltan etiquetar (sin repeticiones) = 391
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1407
Total analizadas actualmente = 3273
Faltan etiquetar 1727 palabras
Porcentaje candidatas = 0.4298808432630614
########################################
Métrica: PalPer
Palabras candidatas = 1541
Total analizadas actualmente = 3533
Faltan etiquetar 146

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [13]:

np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)
df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)

In [14]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [15]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 164
Total analizadas actualmente = 749
Faltan etiquetar 251 palabras
Porcentaje candidatas = 0.21895861148197596
########################################
Métrica: Personas
Palabras candidatas = 214
Total analizadas actualmente = 732
Faltan etiquetar 268 palabras
Porcentaje candidatas = 0.2923497267759563
########################################
Métrica: PalPer
Palabras candidatas = 305
Total analizadas actualmente = 938
Faltan etiquetar 62 palabras
Porcentaje candidatas = 0.3251599147121535
Faltan etiquetar (sin repeticiones) = 457
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1285
Total analizadas actualmente = 3139
Faltan etiquetar 1861 palabras
Porcentaje candidatas = 0.409366040140172
########################################
Métrica: PalPer
Palabras candidatas = 1544
Total analizadas actualmente = 3533
Faltan etiquetar 146